**Initial Regression Analysis**

This is an initial look at specifications for our analysis of maternal mortality and political indicators.

Open questions/issues:
- We may need to get data from 2016 regarding state leadership
- All 50 states are not represented in the maternal mortality data

In [118]:
#setup
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [120]:
#importing the file for maternal mortality rate (MMR) data
mmr_df = pd.read_csv("data/maternal_mortality_complete_dataset.csv")
mmr_df

,Unnamed: 0.1,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,...,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct,State Abbreviation,State,guttmacher_index,state,gop_trifecta
0,0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,...,3.5,71.8,32.1,1.1,3.0,AL,Alabama,most_restrictive,Alabama,1
1,1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,...,19.4,78.3,28.0,1.2,3.3,AL,Alabama,most_restrictive,Alabama,1
2,2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,...,0.0,67.4,41.2,1.1,4.6,AL,Alabama,most_restrictive,Alabama,1
3,3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,...,0.0,66.4,37.5,1.1,3.1,AL,Alabama,most_restrictive,Alabama,1
4,4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,...,3.4,72.3,32.8,1.2,2.9,AL,Alabama,most_restrictive,Alabama,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,3107,4044432,1,Deaths,3712,Sweetwater County (Wyoming),56037.0,1,Total,2,...,14.5,75.2,28.8,1.3,1.1,WY,Wyoming,restrictive,Wyoming,1
3108,3108,4044438,1,Deaths,3697,Teton County (Wyoming),56039.0,1,Total,2,...,70.7,79.3,8.7,0.7,1.0,WY,Wyoming,restrictive,Wyoming,1
3109,3109,4044444,1,Deaths,3714,Uinta County (Wyoming),56041.0,1,Total,2,...,10.0,80.1,25.1,1.0,1.0,WY,Wyoming,restrictive,Wyoming,1
3110,3110,4044450,1,Deaths,3700,Washakie County (Wyoming),56043.0,1,Total,2,...,0.0,76.0,26.0,1.1,1.2,WY,Wyoming,restrictive,Wyoming,1


Question: should we be looking at the lagged data for government control? Two ways of looking at this...

a) we can say that the leadership is a reflection of politics and policy in the state, or 

b) we can use control in 2016 as an indicator of future policy and leadership in the state and the 2017-2019 data is a result of the policies preceeding the data collection.

12/8 update: decided the discrepancy in years is okay

In [122]:
#checking if all states are in the dataset
mmr_df["state"].nunique()

50

In [124]:
#checking the unique values to make sure they are all states
mmr_df["state"].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico',
       'New_York', 'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode_Island', 'South_Carolina',
       'South_Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

I've broken these down into two theories, and then further into 2 or 3 specifications based on those theories.

**First Theory**: A GOP trifecta has a positive relationship with maternal mortality.
- Specification #1: The effect of a GOP trifecta on MMR.
- Specification #2: The effect of GOP trifecta on MMR, standardizing MMR.
- Specification #3: The effect of a GOP trifecta on MMR, controlling for health indicators.

**Second Theory**: The highest level of abortion restriction (Most restrictive, Very restrictive, Restrictive, Some restrictions/protections, Protective, Very protective, and Most protective) corresponds with the highest level of maternal mortality in a state.
- Specification #1:  The effect of the highest level of abortion restriction (only looking at whether a state was in the "most restrictive" category) vs most protective states.
- Specification #2: The effect of all levels of abortion restriction on MMR.
- Specification #3: The effect of all levels of abortion restriction on MMR, controlling for health indicators.

**Specification # 1 (First Theory):** The effect of a GOP trifecta on maternal mortality.

In [126]:
#making a clean copy without NAs in these columns.  This dropped about 30 observations.
mmr_df = mmr_df.dropna(subset=['val', 'gop_trifecta'])

In [128]:
mmr_df

,Unnamed: 0.1,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,...,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct,State Abbreviation,State,guttmacher_index,state,gop_trifecta
0,0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,...,3.5,71.8,32.1,1.1,3.0,AL,Alabama,most_restrictive,Alabama,1
1,1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,...,19.4,78.3,28.0,1.2,3.3,AL,Alabama,most_restrictive,Alabama,1
2,2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,...,0.0,67.4,41.2,1.1,4.6,AL,Alabama,most_restrictive,Alabama,1
3,3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,...,0.0,66.4,37.5,1.1,3.1,AL,Alabama,most_restrictive,Alabama,1
4,4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,...,3.4,72.3,32.8,1.2,2.9,AL,Alabama,most_restrictive,Alabama,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,3107,4044432,1,Deaths,3712,Sweetwater County (Wyoming),56037.0,1,Total,2,...,14.5,75.2,28.8,1.3,1.1,WY,Wyoming,restrictive,Wyoming,1
3108,3108,4044438,1,Deaths,3697,Teton County (Wyoming),56039.0,1,Total,2,...,70.7,79.3,8.7,0.7,1.0,WY,Wyoming,restrictive,Wyoming,1
3109,3109,4044444,1,Deaths,3714,Uinta County (Wyoming),56041.0,1,Total,2,...,10.0,80.1,25.1,1.0,1.0,WY,Wyoming,restrictive,Wyoming,1
3110,3110,4044450,1,Deaths,3700,Washakie County (Wyoming),56043.0,1,Total,2,...,0.0,76.0,26.0,1.1,1.2,WY,Wyoming,restrictive,Wyoming,1


In [130]:
#define y (MMR rate)
y = mmr_df["val"]

#define X (GOP trifecta)
X = mmr_df["gop_trifecta"]

#adding an intercept
X = sm.add_constant(X)

mmr_vs_trifecta_1 = sm.OLS(y,X).fit()

print(mmr_vs_trifecta_1.summary())

                            OLS Regression Results                            
Dep. Variable:                    val   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     191.8
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           2.33e-42
Time:                        11:36:04   Log-Likelihood:                 30234.
No. Observations:                3081   AIC:                        -6.046e+04
Df Residuals:                    3079   BIC:                        -6.045e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         3.144e-05   4.12e-07     76.340   

**Comments:**
- These results are hard to interpret since the values are so small- so solve this, we are standardizing the "val" column (which represents the mortality rate value) for subsequent regressions
- A gop_trifecta seems to have a postive relationship with maternal mortality.
- The results are statistically significant for the intercept and the gop_trifecta coefficients

**Specificaition #2 (First Theory):** The effect of a GOP trifecta on MMR, standardizing MMR.

In [134]:
#calculating mean of MMR
mmr_mean = mmr_df['val'].mean()
#calculating standard deviation of MMR
mmr_std = mmr_df['val'].std()

#creating a new column for standardized MMR
mmr_df['standardized_mmr'] = (mmr_df['val'] - mmr_mean) / mmr_std

/var/folders/n_/qfwwhdvn3xx2_9v1hbk4zhnc0000gq/T/ipykernel_78191/2781902497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mmr_df['standardized_mmr'] = (mmr_df['val'] - mmr_mean) / mmr_std


In [136]:
#define y (MMR rate standardized)
y = mmr_df["standardized_mmr"]

#define X (GOP trifecta)
X = mmr_df["gop_trifecta"]

#adding an intercept
X = sm.add_constant(X)

mmr_vs_trifecta_2 = sm.OLS(y,X).fit()

print(mmr_vs_trifecta_2.summary())

                            OLS Regression Results                            
Dep. Variable:       standardized_mmr   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     191.8
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           2.33e-42
Time:                        11:37:51   Log-Likelihood:                -4278.2
No. Observations:                3081   AIC:                             8560.
Df Residuals:                    3079   BIC:                             8572.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.3404      0.030    -11.286   

**Comments:**
- This specification is MUCH easier to read. It tells us that the gop_trifecta increased maternal mortality by a factor of 0.5126.  Living in a Democratic state is represented by the constant of -0..3404, which indicates a decrease in maternal mortality.
- All results were statistically significant.

**Specification #3 (First Theory):** The effect of a GOP trifecta on MMR, controlling for health indicators.

In [138]:
#was getting NA values in the regression, so needed to filter out the ones I didn't want
na_counts = mmr_df.isnull().sum()
#filter to show only columns with NAs
columns_with_nas = na_counts[na_counts > 0].index.tolist()
print("Columns with NAs:", columns_with_nas)

Columns with NAs: ['county_fips', 'low_income_pct', 'obgyn_provider_rate_100k', 'prenatal_care_first_trimester_pct', 'obesity_pre_pregnancy_pct', 'diabetes_pre_pregnancy_pct', 'hypertension_pre_pregnancy_pct']


In [142]:
#dropping columns with NAs
mmr_df = mmr_df.dropna(subset=['obgyn_provider_rate_100k', 'prenatal_care_first_trimester_pct', 'obesity_pre_pregnancy_pct', 'diabetes_pre_pregnancy_pct', 'hypertension_pre_pregnancy_pct'])

In [144]:
#define y (MMR rate standardized)
y = mmr_df["standardized_mmr"]

#define X (GOP trifecta and health indicator covariates)
X = mmr_df[['gop_trifecta', 'obgyn_provider_rate_100k', 
            'prenatal_care_first_trimester_pct', 'obesity_pre_pregnancy_pct',
            'diabetes_pre_pregnancy_pct', 'hypertension_pre_pregnancy_pct']]

#adding an intercept
X = sm.add_constant(X)

mmr_vs_trifecta_3 = sm.OLS(y,X).fit()

print(mmr_vs_trifecta_3.summary())

                            OLS Regression Results                            
Dep. Variable:       standardized_mmr   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     434.9
Date:                Sun, 08 Dec 2024   Prob (F-statistic):               0.00
Time:                        11:44:59   Log-Likelihood:                -3267.9
No. Observations:                2975   AIC:                             6550.
Df Residuals:                    2968   BIC:                             6592.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

**Comments:**
- GOP Trifecta's effect is slightly smaller taking into account health indicators that may better explain the mortality rate, however, our data still shows that the coefficient for having one is substantively significant, increasing the rate of maternal mortality by 0.31 standard deviations from the mean.
- Few other health indicators appear to be as predictive of maternal mortality, with hypertension before pregnancy coming in second at 0.2773.
- All coefficients were statistically significant.
- Diabetes pre-pregnancy having a negative relationship with maternal mortality was a surprising result. Other health-indicators relationships with maternal mortality were consistent with general knowledge of risk factors of prenancy-related deaths.
- There is potential multicollinearity in our model as mentioned in the notes below the regression.

**Second Theory**: The highest level of abortion restriction (Most restrictive, Very restrictive, Restrictive, Some restrictions/protections, Protective, Very protective, and Most protective) corresponds with the highest level of maternal mortality in a state.
- Specification #1:  The effect of higher levels of abortion restriction (only looking at whether a state was in the "most restrictive" category) vs states that were not in that category on MMR
- Specification #2: The effect of all levels of abortion restriction on MMR
- Specification #3: The effect of all levels of abortion restriction on MMR, controlling for health indicators.

**Specification #1 (Second Theory):** The effect of the highest level of abortion restriction (only looking at whether a state was in the "most restrictive" category) vs most protective states.

In [146]:
#need to first organize into categorical variables using one-hot encoding
#getting the unique columns in guttmacher_index to see what categories will be used
pd.unique(mmr_df['guttmacher_index'])

array(['most_restrictive', 'protective', 'restrictive', 'very_protective',
       'some_restrictions_protections', 'very_restrictive',
       'most_protective'], dtype=object)

In [148]:
#the excluded variable will be "Most Protective." 
mmr_df_dummies = pd.get_dummies(mmr_df, columns = ['guttmacher_index'], drop_first = True, prefix = 'guttmacher')
mmr_df_dummies

,Unnamed: 0.1,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,...,State,state,gop_trifecta,standardized_mmr,guttmacher_most_restrictive,guttmacher_protective,guttmacher_restrictive,guttmacher_some_restrictions_protections,guttmacher_very_protective,guttmacher_very_restrictive
0,0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,...,Alabama,Alabama,1,0.585016,True,False,False,False,False,False
1,1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,...,Alabama,Alabama,1,-0.491689,True,False,False,False,False,False
2,2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,...,Alabama,Alabama,1,2.179015,True,False,False,False,False,False
3,3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,...,Alabama,Alabama,1,2.350452,True,False,False,False,False,False
4,4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,...,Alabama,Alabama,1,0.585599,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,3107,4044432,1,Deaths,3712,Sweetwater County (Wyoming),56037.0,1,Total,2,...,Wyoming,Wyoming,1,-0.280052,False,False,True,False,False,False
3108,3108,4044438,1,Deaths,3697,Teton County (Wyoming),56039.0,1,Total,2,...,Wyoming,Wyoming,1,-1.271855,False,False,True,False,False,False
3109,3109,4044444,1,Deaths,3714,Uinta County (Wyoming),56041.0,1,Total,2,...,Wyoming,Wyoming,1,0.665171,False,False,True,False,False,False
3110,3110,4044450,1,Deaths,3700,Washakie County (Wyoming),56043.0,1,Total,2,...,Wyoming,Wyoming,1,-0.575964,False,False,True,False,False,False


In [150]:
#Need to first convert all the booleans to 1s and 0s
columns_to_convert = [
    'guttmacher_most_restrictive', 'guttmacher_protective', 
    'guttmacher_restrictive', 'guttmacher_some_restrictions_protections', 
    'guttmacher_very_protective', 'guttmacher_very_restrictive'
]

#using the above array, I'll convert the columns using .astype(int), which will turn True/False into 1s and 0s for the regression
mmr_df_dummies[columns_to_convert] = mmr_df_dummies[columns_to_convert].astype(int)
mmr_df_dummies

,Unnamed: 0.1,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,...,State,state,gop_trifecta,standardized_mmr,guttmacher_most_restrictive,guttmacher_protective,guttmacher_restrictive,guttmacher_some_restrictions_protections,guttmacher_very_protective,guttmacher_very_restrictive
0,0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,...,Alabama,Alabama,1,0.585016,1,0,0,0,0,0
1,1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,...,Alabama,Alabama,1,-0.491689,1,0,0,0,0,0
2,2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,...,Alabama,Alabama,1,2.179015,1,0,0,0,0,0
3,3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,...,Alabama,Alabama,1,2.350452,1,0,0,0,0,0
4,4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,...,Alabama,Alabama,1,0.585599,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,3107,4044432,1,Deaths,3712,Sweetwater County (Wyoming),56037.0,1,Total,2,...,Wyoming,Wyoming,1,-0.280052,0,0,1,0,0,0
3108,3108,4044438,1,Deaths,3697,Teton County (Wyoming),56039.0,1,Total,2,...,Wyoming,Wyoming,1,-1.271855,0,0,1,0,0,0
3109,3109,4044444,1,Deaths,3714,Uinta County (Wyoming),56041.0,1,Total,2,...,Wyoming,Wyoming,1,0.665171,0,0,1,0,0,0
3110,3110,4044450,1,Deaths,3700,Washakie County (Wyoming),56043.0,1,Total,2,...,Wyoming,Wyoming,1,-0.575964,0,0,1,0,0,0


In [152]:
#define y (MMR rate standardized)
y = mmr_df_dummies["standardized_mmr"]

#define X (GOP trifecta and health indicator covariates)
X = mmr_df_dummies[['guttmacher_most_restrictive']]

#adding an intercept
X = sm.add_constant(X)

mmr_vs_gutt1 = sm.OLS(y,X).fit()

print(mmr_vs_gutt1.summary())

                            OLS Regression Results                            
Dep. Variable:       standardized_mmr   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     227.0
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           1.70e-49
Time:                        11:50:05   Log-Likelihood:                -4096.8
No. Observations:                2975   AIC:                             8198.
Df Residuals:                    2973   BIC:                             8210.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

Very restrictive is an interesting outlier; need to figure out what other confounding factors might be causing it to have less incidence of maternal mortality.

**Specification #2 (Second Theory):** The effect of all levels of abortion restriction on MMR

In [154]:
#define y (MMR rate standardized)
y = mmr_df_dummies["standardized_mmr"]

#define X (GOP trifecta and health indicator covariates)
X = mmr_df_dummies[['guttmacher_most_restrictive', 
                    'guttmacher_very_restrictive',
                    'guttmacher_restrictive', 
                    'guttmacher_some_restrictions_protections', 
                    'guttmacher_very_protective', 
                    'guttmacher_protective']]

#adding an intercept
X = sm.add_constant(X)

mmr_vs_gutt2 = sm.OLS(y,X).fit()

print(mmr_vs_gutt2.summary())

                            OLS Regression Results                            
Dep. Variable:       standardized_mmr   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     83.20
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           1.49e-96
Time:                        11:50:08   Log-Likelihood:                -3975.1
No. Observations:                2975   AIC:                             7964.
Df Residuals:                    2968   BIC:                             8006.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

Very restrictive is again an interesting outlier...

**Specification #3 (Second Theory):** The effect of all levels of abortion restriction on MMR, controlling for health indicators.

In [156]:
#define y (MMR rate standardized)
y = mmr_df_dummies["standardized_mmr"]

#define X (GOP trifecta and health indicator covariates)
X = mmr_df_dummies[['guttmacher_most_restrictive', 
                    'guttmacher_very_restrictive',
                    'guttmacher_restrictive', 
                    'guttmacher_some_restrictions_protections', 
                    'guttmacher_protective',
                    'guttmacher_very_protective', 
                    'obgyn_provider_rate_100k', 
                    'prenatal_care_first_trimester_pct', 
                    'obesity_pre_pregnancy_pct', 
                    'diabetes_pre_pregnancy_pct', 
                    'hypertension_pre_pregnancy_pct']]

#adding an intercept
X = sm.add_constant(X)

mmr_vs_gutt3 = sm.OLS(y,X).fit()

print(mmr_vs_gutt3.summary())

                            OLS Regression Results                            
Dep. Variable:       standardized_mmr   R-squared:                       0.483
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     251.2
Date:                Sun, 08 Dec 2024   Prob (F-statistic):               0.00
Time:                        11:50:35   Log-Likelihood:                -3226.3
No. Observations:                2975   AIC:                             6477.
Df Residuals:                    2963   BIC:                             6549.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

**Comments:**
- All were statistically significant except for the constant (baseline indicator for the most protective category)
- Potential multicollinearity as in the above regression
- Diabetes seemed to decrease maternal mortality for some reason, but for all other health indicators it seems to align with our hypothesis as in the first regression

Interestingly, when correcting for health indicators, maternal mortality seems to be a little more nuanced than the common story is. We expected to see an increase in maternal mortality as restrictions were tighter, and while that's true, maternal mortality is higher in states that are protective of abortion restrictions than the ones that are not protective. 

When considering all the results, it appears a GOP trifecta is more predictive of maternal mortality when compared to how tight policy restrictions/protections are in a state. This could be because the executive branch (i.e. governorship) has more influence on how the laws are carried out in states with a GOP trifecta.

In [162]:
#aggregate maternal mortality rates by state
state_mmr = mmr_df.groupby('state')['val'].mean().reset_index()

#rename columns for clarity
state_mmr.columns = ['State', 'Maternal Mortality Rate']

#sort by maternal mortality rate
state_mmr.sort_values(by='Maternal Mortality Rate', ascending = False, inplace=True)

#display the results
print(state_mmr)

             State  Maternal Mortality Rate
23     Mississippi                 0.000060
17       Louisiana                 0.000054
9          Georgia                 0.000054
39  South_Carolina                 0.000052
0          Alabama                 0.000051
3         Arkansas                 0.000044
35        Oklahoma                 0.000044
32  North_Carolina                 0.000043
1           Alaska                 0.000039
7         Delaware                 0.000039
8          Florida                 0.000038
13         Indiana                 0.000038
41       Tennessee                 0.000038
16        Kentucky                 0.000038
42           Texas                 0.000037
33    North_Dakota                 0.000037
40    South_Dakota                 0.000037
24        Missouri                 0.000037
34            Ohio                 0.000035
2          Arizona                 0.000035
30      New_Mexico                 0.000034
15          Kansas              

Ideas for visualizations:
-List of each policy and state (I'll break this down further in the paper and talk about specific laws)
-The most/least restrictive states in terms of maternal mortality and which states had the highest maternal mortality (I included a matrix above with the maternal mortality info)